### Processing the DAVIS and KIBA datasets
#### <span style="color:blue">This notebook is for processing the data to obtain proteins whose length<\=1024 and accordingly remove binding interactions from the dataset with protein targets above 1024 sequence length.</span>

In [ ]:
import os
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle,urllib
import ast,json

### Select the dataset and provide the path to "proteins.txt" file

In [ ]:
file=open("/data/davis/proteins.txt","r")
contents=file.read()
protein_data=ast.literal_eval(contents)
protein_data.keys()

### From the available proteins, select the proteins<=1024 and save.

In [ ]:
upd_protein_data = dict()
for (key, value) in protein_data.items():
    if len(value)<=1024:
        upd_protein_data[key] = value
print(upd_protein_data)
f = open("/data/davis/proteins_updated.txt","w")
f.write( str(upd_protein_data) )
f.close()

### Remove binding interactions with protein targets >1024 and save the groundtruth file of the updated data.

In [ ]:
Y=pickle.load(open("/data/davis/Y", "rb"), encoding='latin1')
list_keys= list(set(protein_data.keys()) - set(upd_protein_data.keys()))
df = pd.DataFrame(Y,  columns=list(protein_data.keys()))
df2=df.drop(list_keys, axis = 1)
#df.to_csv('/data/davis/davis.csv', index=False) #original
df2.to_csv('/data/davis/davis_updated.csv', index=False) #updates davis dataset
Y_updated=df2.to_numpy()
np.save('/data/davis/Y_updated.npy',Y_updated)

### Divide the available data into 6 parts- 5 for training+validation and 1 for testing

In [ ]:
data_full=np.where(np.isnan(Y_updated.flatten())==False)[0]#Total interactions
samples=[i for i in range(0,data_full)]
def partition(list_in,n):
    random.shuffle(list_in)
    return [list_in[i::n] for i in range(n)]
parts_6=partition(samples,6)
with open('/data/davis/folds/train_fold_setting2.txt','w') as train_file: #saving the 5 parts as training fold
    json.dump(parts_6[0:5],train_file)
with open('/Users/rohan/Desktop/DTI_setup/data/davis/folds/test_fold_setting2.txt','w') as test_file: #test fold
    json.dump(parts_6[5],test_file)
